In [31]:
import nltk
import string
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

In [32]:
df = pd.read_csv("spam_dataset.csv")
df.head(5)

,label,text
0,spam,Your internet speed is slow! Upgrade now for f...
1,spam,Don't miss out on our flash sale. Prices slash...
2,not spam,Happy to have you as a colleague. Your contrib...
3,not spam,Thanks for your help in organizing the team-bu...
4,spam,Claim your free trial of our language learning...


In [33]:
encoder = LabelEncoder()
df["label"] = encoder.fit_transform(df["label"])

In [34]:
df.head(5)

,label,text
0,1,Your internet speed is slow! Upgrade now for f...
1,1,Don't miss out on our flash sale. Prices slash...
2,0,Happy to have you as a colleague. Your contrib...
3,0,Thanks for your help in organizing the team-bu...
4,1,Claim your free trial of our language learning...


In [35]:
df.shape

(500, 2)

In [36]:
df = df.drop_duplicates()
df.shape

(216, 2)

In [37]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [38]:
def get_importantFeatures(sent):
    sent = sent.lower()
    
    returnList = []
    sent = nltk.word_tokenize(sent)
    for i in sent:
        if i.isalnum():
            returnList.append(i)
    return returnList

def removing_stopWords(sent):
    returnList = []
    for i in sent:
        if i not in nltk.corpus.stopwords.words('english') and i not in string.punctuation:
            returnList.append(i)
    return returnList

def potter_stem(sent):
    returnList = []
    for i in sent:
        returnList.append(ps.stem(i))
    return " ".join(returnList)

In [39]:
df['imp_feature'] = df['text'].apply(get_importantFeatures)
df['imp_feature'] = df['imp_feature'].apply(removing_stopWords)
df['imp_feature'] = df['imp_feature'].apply(potter_stem)

In [40]:
df

,label,text,imp_feature
0,1,Your internet speed is slow! Upgrade now for f...,internet speed slow upgrad faster brows
1,1,Don't miss out on our flash sale. Prices slash...,miss flash sale price slash product
2,0,Happy to have you as a colleague. Your contrib...,happi colleagu contribut invalu
3,0,Thanks for your help in organizing the team-bu...,thank help organ event
4,1,Claim your free trial of our language learning...,claim free trial languag learn app speak fluen...
...,...,...,...
475,0,Let's plan a team lunch to celebrate recent ac...,let plan team lunch celebr recent achiev
478,0,Your positive attitude in the workplace is con...,posit attitud workplac contagi keep
482,1,Special offer for subscribers. Get a free tria...,special offer subscrib get free trial stream s...
491,0,The new office decor looks fantastic. Kudos to...,new offic decor look fantast kudo team


In [42]:


from sklearn.model_selection import train_test_split

x = df["imp_feature"]
y = df["label"]

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size =30,random_state = 42)


In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
feature = tfidf.fit_transform(x_train)

tuned_parameters = {'kernel':['linear','rbf'],'gamma':[1e-3,1e-4], 'C':[1,10,100,1000]}

model = GridSearchCV(svm.SVC(),tuned_parameters)
model.fit(feature, y_train)



GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                         'kernel': ['linear', 'rbf']})

In [48]:
# Save the TfidfVectorizer to a pickle file
tfidf_filename = 'tfidf_vectorizer.pkl'
with open(tfidf_filename, 'wb') as file:
    pickle.dump(tfidf, file)

In [45]:
y_predict = tfidf.transform(x_test)
print("Accuracy:",model.score(y_predict,y_test))

Accuracy: 0.9666666666666667


In [47]:

import pickle
filename = 'finalspammsg_detector_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
with open('tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(tfidf_vectorizer, file)